In [1]:
import os
import re
import numpy as np

In [2]:
logfile_path = '../../ISWC2022LearnableFilter/Run_logs/logs/'

In [3]:
experiments = os.listdir(logfile_path)

# average over subjects

In [4]:
for e in experiments:
    experiment_path = logfile_path + e
    score_path = experiment_path + '/finetuned_score.txt'
    if not os.path.exists(score_path):
        score_path = experiment_path + '/score.txt'
    if os.path.exists(score_path):
        print(f'Processing {e}...')
        
        weighted_F1 = []
        macro_F1 = []
        
        f = open(score_path)
        while True:
            line = f.readline()
            if line == '':
                break
            scores = re.findall(r'0.[\d.]*\d+', line)
            weighted_F1.append(float(scores[0]))
            macro_F1.append(float(scores[1]))
        
        F1 = np.array([weighted_F1, macro_F1]).T
        mean_F1 = np.mean(F1, axis=0)
        np.savetxt(experiment_path+'/average_score.txt', mean_F1, fmt='%.6f')

Processing deepconvlstm_data_skodar_seed_3_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_skodar_seed_4_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_skodar_seed_2_windowsize_84_waveFilter_True_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_True_wavelearnble_False...
Processing deepconvlstm_data_skodar_seed_1_windowsize_84_waveFilter_True_Fscaling_0.5_cvfilter_64_lstmfilter_128_Regu_True_wavelearnble_False...
Processing deepconvlstm_data_skodar_seed_5_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_pamap2_seed_5_windowsize_168_waveFilter_True_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_True_wavelearnble_False...
Processing sahar_data_pamap2_seed_1_windowsize_168_waveFilter_True_Fscaling_0.25_cvfilter_128_grufilter_None_Regu_True_wa

# Average over seeds

In [5]:
# models = ['deepconvlstm', 'attend', 'sahar']
# datasets = ['oppo', 'hapt', 'skodar', 'pamap2', 'dsads']
# wavelets = ['False', 'True', 'None']
# sfs = ['1']

In [6]:
models = ['deepconvlstm']
datasets = ['pamap2',]
wavelets = ['False']
sfs = [ '0.75']

In [7]:
for model in models:
    for dataset in datasets:
        for wavelet in wavelets:
            for sf in sfs:
                score_cross_seed = []
                for e in experiments:
                    if e.endswith('e'):
                        this_model = re.findall(r'(.+?)_data', e)[0]
                        this_scaling = re.findall(r'Fscaling_(.+?)_cvfilter', e)[0]
                        this_dataset = re.findall(r'_data_(.+?)_seed', e)[0]
                        this_wavelet = re.findall(r'wavelearnble_(.+)', e)[0]
            
                        if (this_model==model) and (this_dataset==dataset) and (this_scaling==sf) and (this_wavelet==wavelet):              
                            seed_path = logfile_path + e + '/average_score.txt'
                            score = np.loadtxt(seed_path)
                            score_cross_seed.append(score)
                print(score_cross_seed)
                m = np.mean(score_cross_seed, axis=0)
                std = np.std(score_cross_seed, axis=0)
                result_cross_seed = np.array([m, std])
                np.savetxt(f'{logfile_path}/../result/{model}_{dataset}_{sf}_{wavelet}.txt', result_cross_seed, fmt='%.6f')

[array([0.863797, 0.7716  ]), array([0.858173, 0.761202]), array([0.850396, 0.742724]), array([0.813478, 0.756146]), array([0.842152, 0.743322])]
